In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Objective**

1. Techniques to Feature Engineering

2. Thorough Exploratory Data Analysis (EDA)

**Introduction**

This is my 2nd Kernel so still learning! Do help by commenting any improvements you feel i could use! This is based on the Give Me Some Credit : 2011 Competition Data (Classification problem type). I'm mainly self-taught so don't mind the basic or 'greenhorn' approaches used. 

> Do note that this Kernel focuses on Feature Engineering & EDA. Hence, the explanations are emphasized more on these 2 aspects (Chapters 3 to 8). In terms of explanations, I've also included the logic for each feature engineered & pattern evaluations during EDA. 

>Now lets Dive in!


**Chapter Outline**

0.Dataset Dictionary (For Reference only)

1.Open Dataset

2.Preliminary Overview

    2.1 Shape
    2.2 Head raw preview
    2.3 Sample raw preview
    2.4 Brief Info
    2.5 Describe (Max, Min, Quartiles)
    2.6 Own Quick-Stats Function

3.Preliminary Data Cleaning [C1-Correct, C2-Complete]

    3.1 Age
    3.2 Monthly Income
    3.3 Number of Dependents

4.Brief EDA

    4.1 Target Variable
    4.2 Other Features
    4.3 Notes for Feature Engineering

5.Deeper Data Cleaning/Feature Engineering [C3-Create, C4-Convert]

    5.1 Group_A (Combine Feature)
    5.2 Group_B (New Feature Net Worth)
    5.3 Group_C (Combine Feature)
    5.4 New Feature (Debt Payments)
    5.5 New Feature (Age Category)
    5.6 New Feature (Social Economic Status SES Category)
    5.7 Checks again before Charting for EDA

6.Deeper EDA

    6.1 Quick Data Prep
    6.2 Uni-variate
    6.3 Bi-variate

7.EDA Commentary

    7.1 Summative
    7.2 Heat-Map Reference

8.Final Feature Engineering Commentary

9.Prepare Data - A

    9.1 Establish Features Decided on (Keep/Discard)
    9.2 Check Heat-Map Again
    9.3 Replicate for Test-set (DataCleaning & FeaturesDecided)

10.Quick Feature Engineering Validation

    10.1 Accuracy Score (Without Preliminary Engineering)
    10.2 Accuracy Score (With Preliminary Engineering)

11.Prepare Data - B

    11.1 Train/Test Split
    11.2 Feature Comparison Function
    11.3 LASSO L1 Regularized
    11.4 Ridge L2 Regularized
    11.5 Logisitc Regression Balanced by sample weight
    11.6 XGBoost Classifier
    11.7 Random Forest Classifier

12.Features (Side To Side Comparison)

    12.1 (Coefficient Values) Quick Easy Method
    12.2 (Coefficient Values) Neat DataFrame Method
    12.3 (Plotting) Quick Method
    12.4 (Plotting) Sorted Neat Method
    12.5 Quick Commentary Result (Feature Engineering VS Importance)

13.ROC AUC (Side To Side Comparison)

14.Cross Validation (Side To Side Comparison)


# Summative of EDA

**Dataset:**

    - Features has extreme frequency’s concentrated within sub-categories
    - High Kurtosis & Skewness of some features
    - Huge income disparity
    - Predominant retired (>63yrs old) dataset
    - Unbalanced target datases

**Trends:**
    
    - Those who have had experience financial distress (target variable)
    >Made lesser loans or exceed deadlines
    >Tend to have lesser dependents & debt ratio & net worth
    >As expected are of lower-tier income, But lower debt ratio

    - Ignoring mortality and time value of money (i.e.Annuities)
    >Debt ratio & Net worth shows gaussian distribution against age

    - Those who had acts of debt delinquency (Made loans or exceed deadlines)
    >Tend to be from the higher-tier income or Retired

    -Others
    >The higher the income, the higher the debt ratio
    >The higher the income, the lower the dependents


# **Some Considerations :**
 
    -Feature Engineering
    A.Consider context of problem
    B.Consier dispersion/balance of dataset
    C.Avoid disrupting the inherent distribution
    D.Use intuitive logical reasoning when imporvising benchmarks (e.g.Minimum Age, Income to age dispersion)
    E.Go beyong context (E.g.General Net Worth formula used by broad pension-type valuations)
    -EDA
    F.Have alternative charts for high Kurtosis & Skewed data.
    G.Use multiple references when discarding engineered featurs (e.g.Distribution charts & HeatMap correlations )


# 0.Dataset Dictionary (For Reference only)

Feature Name: Description {Type}

* SeriousDlqin2yrs: Person experienced 90 days past due delinquency or worse {Y/N}

* RevolvingUtilizationOfUnsecuredLines: Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits {percentage}
                                        
* age: Age of borrower in years {integer}

* NumberOfTime30-59DaysPastDueNotWorse: Number of times borrower has been 30-59 days past due but no worse in the last 2 years. {integer}
                                        
* DebtRatio: Monthly debt payments, alimony,living costs divided by monthly gross income {percentage}
                                        
* MonthlyIncome: Monthly income {real}

* NumberOfOpenCreditLinesAndLoans: Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards) {integer}
                                        
* NumberOfTimes90DaysLate: Number of times borrower has been 90 days or more past due.	{integer}

* NumberRealEstateLoansOrLines: Number of mortgage and real estate loans including home equity lines of credit {integer}
                                        
* NumberOfTime60-89DaysPastDueNotWorse: Number of times borrower has been 60-89 days past due but no worse in the last 2 years. {integer}
                                        
* NumberOfDependents: Number of dependents in family excluding themselves (spouse, children etc.) {integer}

# 1.Open Dataset

Lets first open the dataset!!

In [1]:
# Import Modules

# Foundational Packages
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.display.max_columns = 100

In [2]:
"""Read & Open CSV files"""
# Open Train & Test files
train_raw = pd.read_csv('../input/cs-training.csv', na_values=-1) #FYI na_values are defined in the original data page
test_raw = pd.read_csv('../input/cs-test.csv', na_values=-1)
# Copy Train file for workings
train_raw_copy = train_raw.copy(deep=True)

# **2.Preliminary Overview**

Now to do some quick overview. In short the shape for the structure which simply is the size dimensions (length of rows and width of columns) of the data-set.

Head & Samples are to give us a authentic visual of the dataset.

Info & Describes gives us the computational datatypes & basic statistics respectively.

# **2.1** Shape

In [3]:
"""Shape"""
print('Train Shape: ', train_raw_copy.shape)
print('Test Shape: ', test_raw.shape)

# **2.2** Head

*** Train set**

In [4]:
display(train_raw_copy.head())

*** Test set**

In [5]:
display(test_raw.head())

# **2.3** Sample

The difference between head & sample is that sample will show you a random quantity. while head takes the first defaulted 5 rows of data.

*** Train set**

In [6]:
SampleN = 10
display(train_raw_copy.sample(SampleN))

*** Test set**

In [7]:
SampleN = 10
display(test_raw.sample(SampleN))

# **2.4** Brief Info

Note that I have hid the outputs here as my function at **2.6** is a better inbuilt replacement function instead.

*** Train set** *Unhide to view output

In [8]:
print(train_raw_copy.info())

 * ** Test set ** *Unhide to view output

In [9]:
print(test_raw.info())

# **2.5** Describe (Max Min Quartiles)

* ** Train set**

In [10]:
display(train_raw_copy.describe())

* ** Test set**

In [11]:
display(test_raw.describe())

# **2.6** Own Quick-Stats Check Function

Now for a further stats on the actual components of each respective feature.

Going forward, I will always use this function as a "Check" measure everytime I conduct cleaning/feature engineer on the dataset.

In [12]:
"""Quick Stats: Data Types"""
# Function to output missing values & UniqueCounts & DataTypes
def basic_details(df):
    details = pd.DataFrame()
    details['Missing value'] = df.isnull().sum()
    details['N unique value'] = df.nunique()
    details['dtype'] = df.dtypes
    display(details)

* ** Train set**

In [13]:
basic_details(train_raw_copy)

* ** Test set**

In [14]:
basic_details(test_raw)

Quick Commentary:

**GREAT!!!** Fortunately, we only need to clean up 3x features (age & MonthlyIncome & NumberOfDependents)

**Reasoning:**

Age: Not sensible to have a Minimum age of 0.

MonthlyIncome: 20103 Missing values

NumberOfDependents: 2626 Missing values


# **3.Preliminary Data Cleaning**

Now we know what data-set ‘’Has’’ and ‘’Lacks’’, we proceed to tidy these imperfections.. Here we will mainly execute C1-Correct & C2-Complete.

**C1-Correct**

Where we will correct the dataset by removing any uneccesary segments of the data. The objective is to remove unnecessary noise (prediction errors) and computational costs (run time of code sequence) to the analysis and modelling process as we proceed.

**C2-Complete**

Where we will complete the dataset by filling up any missing values. The most common approach is to replace it with either the mode/mean/median.

I will explain further as we proceed on...

# 3.1 Age

I've pasted the basic statistics found. Reason for this is that we will try our best to align any "Data Cleaning" approaches to the current age dispersion. Thereby not disrupting the natural state (distribution) of the age range in the dataset.

I've assumed the legal age of 18 since this is in a financial context. In addition, since it was only 1 case I've decided to replace that single 0 age occurance with the median.

Subsequently, "brute-forcing" by converting the datatype into integer to reduce computational cost & charting use later on. 

* ** Check those below threshold**

In [15]:
"""Age"""
# Reason due to Age having 0 as minimum which is not sensible
# Decided to input Median

# From above info we found these Age Stats:
# Train                 Test
# Mean - 52.30          Mean - 52.41
# Max - 109             Max - 104
# 75% - 63              75% - 63
# 50% - 52              50% - 52
# 25% - 41              25% - 41
# Min - 0               Min - 21

# Find those less than assumed legal age 18yrs old
LegalAge = 18
LessLegalAgeCount = len(train_raw_copy.loc[train_raw_copy["age"] < 18, "age"])
print("Total number of less than assumed legal age {} is {}".format(LegalAge, LessLegalAgeCount))

Ok we only have 1 . Now lets replace it & ensure it aligns to the correct computational data-type

* ** Replace**

In [16]:
# Replace with Median
train_raw_copy.loc[train_raw_copy["age"] == 0, "age"] = train_raw_copy.age.median()

# Convert data-type
train_raw_copy["age"] = train_raw_copy["age"].astype('int64')

* ** Check again**

Now lets check it again

In [17]:
"""Check Again"""
print(train_raw_copy["age"].describe())

# 3.2 **Monthly Income**

Since we are in a financial context, realistically speaking income does vary proportionately along with age groups. Unless you are MarkZuckerberg or BillGates...

Based on the basic stats derived earlier during the Preliminary Analysis of each Quartile 25% 50% 75% (**2.5 Describe (Max Min Quartiles)**), I've decided on 3x categories to hopefully not disrupt the age dispersion too much. In addition, having been in a pensions consultancy placement at the moment actuaries often assume retirement ages of between 60 to 65.

Age_Range_1: Between ages 18 inclusive to 41 excluding (Working)
Age_Range_2: Between ages 41 inclusive to 63 excluding (Senior Working)
Age_Range_3: Above 63 including (Retired)

* ** We first derive the respective range mean incomes as mentioned above**

In [18]:
# Reason due to Monthly Income having 29731 Missing Values
# Decided to input Median per quartile Age range

# Determine/Set rows that fulfil these quartile range conditions
Age_Range_1 = train_raw_copy.loc[(train_raw_copy["age"] >= 18) & (train_raw_copy["age"] < 41)]
Age_Range_2 = train_raw_copy.loc[(train_raw_copy["age"] >= 41) & (train_raw_copy["age"] < 63)]
Age_Range_3 = train_raw_copy.loc[(train_raw_copy["age"] >= 63)]

# Per Determine/Set rows, Find that range mean MonthlyIncome
Age_R1_MonthlyIncome_impute = Age_Range_1.MonthlyIncome.mean()
Age_R2_MonthlyIncome_impute = Age_Range_2.MonthlyIncome.mean()
Age_R3_MonthlyIncome_impute = Age_Range_3.MonthlyIncome.mean()

* ** Now to fill them**

In [19]:
# Fill Missing MonthlyIncome with 99999 for easy reference
train_raw_copy["MonthlyIncome"] = train_raw_copy["MonthlyIncome"].fillna(99999)
# Convert into integer dtype
train_raw_copy["MonthlyIncome"] = train_raw_copy["MonthlyIncome"].astype('int64')

# Now to fill them
train_raw_copy.loc[(train_raw_copy["age"] >= 18) & (train_raw_copy["age"] < 41) & (train_raw_copy["age"] == 99999)] = Age_R1_MonthlyIncome_impute

train_raw_copy.loc[(train_raw_copy["age"] >= 41) & (train_raw_copy["age"] < 63) & (train_raw_copy["age"] == 99999)] = Age_R2_MonthlyIncome_impute

train_raw_copy.loc[(train_raw_copy["age"] >= 63) & (train_raw_copy["age"] == 99999)] = Age_R3_MonthlyIncome_impute

* **Check again**

In [20]:
"""Check Again"""
basic_details(train_raw_copy)

# 3.3 **Number of Dependents**



This part is pretty straightforward...As intuitive as it is I've decided to input 0, since it makes sense to not have any spouse or family

* **Fill**

In [21]:
# Reason due to NumberOfDependents having 3924 Missing Values

# Fill missing with zero's
train_raw_copy["NumberOfDependents"] = train_raw_copy["NumberOfDependents"].fillna(0)
# Convert into integer dtype
train_raw_copy["NumberOfDependents"] = train_raw_copy["NumberOfDependents"].astype('int64')

# Check counts per category of 'NumberOfDependents'
#print(train_raw_copy.NumberOfDependents.value_counts())

* **Check again**

In [22]:
"""Check Again"""
basic_details(train_raw_copy)

**Data Cleaning Done!!!**

# **4.Brief EDA**

Now to get a quick EDA on the dataset before we perform further 'Data Cleaning'

# 4.1 Target Variable

**SeriousDlqin2yrs (i.e.Target Variable)**
Lets first see the balance of the dataset

* **Proportions**

In [23]:
"""SeriousDlqin2yrs (i.e.Target Variable)"""
print("Exploring SeriousDlqin2yrs (i.e.Target Variable)...")

# List Comprehension
class_0 = [c for c in train_raw_copy['SeriousDlqin2yrs'] if c == 0]
class_1 = [c for c in train_raw_copy['SeriousDlqin2yrs'] if c == 1]
# # Alternative Mask Method
# class_0 = train_raw_copy.SeriousDlqin2yrs.value_counts()[0]
# class_1 = train_raw_copy.SeriousDlqin2yrs.value_counts()[1]

class_0_count = len(class_0)
class_1_count = len(class_1)

print("Target Variable Balance...")
print("Total number of class_0: {}".format(class_0_count))
print("Total number of class_1: {}".format(class_1_count))
print("Occurance event rate: {} %".format(round(class_1_count/(class_0_count+class_1_count) * 100, 3)))   # round 3.dp

* **Plotting**

In [24]:
# Plot
sns.countplot("SeriousDlqin2yrs", data=train_raw_copy)
plt.show()

Right...we have a unbalanced dataset here. In short, since we have a higher proportion of the dataset not having experienced financial distress (SeriousDlqin2yrs=0) our model may overfit easily to this rather than accurately predicting those having experienced financial distress. Of which is our model objective....Consequently, this would also likely cause an inaccurate portray of the evaluation metrics itself (ROC AUC)...

At the mean time, we will just proceed with this until we see the actual model accuracy results. Do note that I will also be doing a subsequent Kernel focusing more on "Data Cleaning" to curb issues similar to this. This is my 2nd Kernel and I'm still learning! So apologies in advance!

# 4.2 Other Features 

* **HeatMap**

We will now use the HeatMap to see which features we can discard or merge.

The **Objective** here is to remove those features bearing high correlations (Remove Multi-Colinearity which causes over-fitting) & engineer more features to optimize the predictive model

In [25]:
"""Correlation Heat-Map"""
cor = train_raw_copy.corr()
plt.figure(figsize=(10, 8))
# sns.set(font_scale=0.7)
sns.heatmap(cor, annot=True, cmap='YlGn')
plt.show()

# 4.3 Notes for Feature Engineering

* **Group_A**: Bears high inter-correlations (0.98)

NumberOfTime30-59DaysPastDueNotWorse,  

NumberOfTimes90DaysLate, 

NumberOfTime60-89DaysPastDueNotWorse

* **Group_B**: Suprisingly low correaltion to Target  Variable  (-0.026)

MonthlyIncome

* **Group_C**: Bears high inter-correlations (0.43)

NumberOfOpenCreditLinesAndLoans, 

NumberRealEstateLoansOrLines

# **5.Deeper Data Cleaning/Feature Engineering [C3-Create, C4-Convert]**

Now with a good reference point we can further execute feature engineering

# 5.1 Group_A

* **Quantity of Defaults**

NumberOfTime30-59DaysPastDueNotWorse, 

NumberOfTimes90DaysLate, 

NumberOfTime60-89DaysPastDueNotWorse

Evidently, we can see they all bear high correlations 0.98. We will try to merge them to avoid collinearity which may overfit the model by distorting or giving higher and incorrect feature weightings.

* **General Idea**

Create a new dataframe column of a Binary statistical datatype indicating if a default was made before or not. (1/0) (Yes/No)

In [26]:
"""
Clean Data - 3 - Creating
"""

"""Joining No# Times past due: CombinedDefault"""
# Reason huge strong correlations i.e.0.98 & 0.99
# Decided to Sum all & Change into Binary Feature (1/0) (Y/0)
# Create Dummy Reference df
train_raw_copy['CD'] = (train_raw_copy['NumberOfTime30-59DaysPastDueNotWorse']
                                     + train_raw_copy['NumberOfTimes90DaysLate']
                                     + train_raw_copy['NumberOfTime60-89DaysPastDueNotWorse'])

# Set '1' for those more than zero indicating Yes there was a default before
train_raw_copy['CombinedDefault'] = 1
train_raw_copy.loc[(train_raw_copy['CD'] == 0), 'CombinedDefault'] = 0
# Remove Dummy Reference df
del train_raw_copy['CD']

# 5.2 Group_B (New Feature Net Worth)

* **NEW FEATURE: Net Worth**

Monthly Income seems to have little correlation (-0.026). However, intutively speaking this should have a strong correlation to financial distress.

* **General Idea**

We will now use a very generic networth formula. In short, incorporate Age & Monthly Income to derive Net Worth


In [27]:
"""New Feature: Net Worth"""
# Decided on general formula NetWorth = (MonthlyIncome x Age) / 10
# https://www.bogleheads.org/forum/viewtopic.php?t=195357
NetWorthDivisor = 10
train_raw_copy['NetWorth'] = train_raw_copy['MonthlyIncome'] * train_raw_copy['age'] / NetWorthDivisor

# 5.3 Group_C

* **Quantity of Loans made**

NumberOfOpenCreditLinesAndLoans, 

NumberRealEstateLoansOrLines

Similar to quantity of defaults, we can see they too high correlations 0.43. We will try to merge them now.

* **General Idea**

Same as **Quantity of Defaults** we will do a Binary Yes or No.



In [28]:
"""Join No# Loans: CombinedLoan"""
# Reason huge strong correlations i.e.0.43
# Decided to Sum all & WITH BUFFER of 5times
LoanLinesBuffer = 5
# Create Dummy Reference df
train_raw_copy['CL'] = train_raw_copy['NumberOfOpenCreditLinesAndLoans'] + train_raw_copy['NumberRealEstateLoansOrLines']

train_raw_copy['CombinedLoan'] = 1
train_raw_copy.loc[train_raw_copy['CL'] >= LoanLinesBuffer, 'CombinedLoan'] = 1
train_raw_copy.loc[train_raw_copy['CL'] < LoanLinesBuffer, 'CombinedLoan'] = 0
# Remove Dummy Reference df
del train_raw_copy['CL']

# 5.4 New Feature (Debt Payments)

* **NEW FEATURE: Debt Payments**

Since we are given components of the ratio, why not? It's always better to do extra than not!



In [29]:
"""New Feature: Monthly debt payments"""
# Derivative formula MonthlyDebtPayments = (DebtRatio) x (MonthlyIncome)
train_raw_copy['MonthlyDebtPayments'] = train_raw_copy['DebtRatio'] * train_raw_copy['MonthlyIncome']
train_raw_copy['MonthlyDebtPayments'] = train_raw_copy['MonthlyDebtPayments'].astype('int64')

# 5.5 New Feature (Age Category)

* **NEW FEATURE: Age Category**

Since we used this approach to execute C2-Complete missing MonthlyIncome values, we shall continue and now include this categorical feature in.



In [30]:
"""New Feature: Age Category"""
train_raw_copy["Age_Map"] = train_raw_copy["age"]
train_raw_copy.loc[(train_raw_copy["age"] >= 18) & (train_raw_copy["age"] < 41), "Age_Map"] = 1
train_raw_copy.loc[(train_raw_copy["age"] >= 41) & (train_raw_copy["age"] < 63), "Age_Map"] = 2
train_raw_copy.loc[(train_raw_copy["age"] >= 63), "Age_Map"] = 3

# replacing those numbers to categorical features then get the dummy variables
train_raw_copy["Age_Map"] = train_raw_copy["Age_Map"].replace(1, "Working")
train_raw_copy["Age_Map"] = train_raw_copy["Age_Map"].replace(2, "Senior")
train_raw_copy["Age_Map"] = train_raw_copy["Age_Map"].replace(3, "Retired")

train_raw_copy = pd.concat([train_raw_copy, pd.get_dummies(train_raw_copy.Age_Map, prefix='is')], axis=1)

# 5.6 New Feature (SES Category)

* **NEW FEATURE: Age Category**

Reason being we will use this to gain a deeper insight during EDA. 

Rather than just knowing the general trend of age, we also want to know which category of age bears the trend too. In other words, the trend for each sub-SES category


In [31]:
"""New Feature: Income Category"""
train_raw_copy["Income_Map"] = train_raw_copy["MonthlyIncome"]
train_raw_copy.loc[(train_raw_copy["MonthlyIncome"] <= 3400), "Income_Map"] = 1
train_raw_copy.loc[(train_raw_copy["MonthlyIncome"] > 3400) & (train_raw_copy["MonthlyIncome"] <= 8200), "Income_Map"] = 2
train_raw_copy.loc[(train_raw_copy["MonthlyIncome"] > 8200), "Income_Map"] = 3

# replacing those numbers to categorical features then get the dummy variables
train_raw_copy["Income_Map"] = train_raw_copy["Income_Map"].replace(1, "LowY")
train_raw_copy["Income_Map"] = train_raw_copy["Income_Map"].replace(2, "MidY")
train_raw_copy["Income_Map"] = train_raw_copy["Income_Map"].replace(3, "HighY")

train_raw_copy = pd.concat([train_raw_copy, pd.get_dummies(train_raw_copy.Income_Map, prefix='is')], axis=1)


# 5.7 **Checks again before Charting for EDA**

Final checks before doing any deeper EDA!

* Checks

In [32]:
"""Check Again"""
basic_details(train_raw_copy)

* Just a raw truncated peek into what we have engineered thus far

In [33]:
SampleN = 15
display(train_raw_copy.sample(SampleN))

# **6.Deeper EDA**

Now with a cleaned up dataset we can start doing EDA for gain a deep insight into what data we are modelling. 

Specifically, EDA helps us to get a clearer idea of what relationships or abnormal relationships we have from the features. These may include outliers, a skewed data, reasonableness checks, feature selection etc.

# 6.1 **Quick Data Prep**

Lets set our variables for easy referencing! *Unhide to view output

In [34]:
"""
Prepare Data for Charting
"""
Features_Preselect_All = train_raw_copy.columns
Features_Preselect_Original = ['SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age',
                               'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
                               'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
                               'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
                               'NumberOfDependents']
Features_Preselect_Engineered = ['CombinedDefault', 'NetWorth', 'CombinedLoan', 'MonthlyDebtPayments', 'is_Retired',
                                 'is_Senior', 'is_Working', 'is_HighY', 'is_LowY', 'is_MidY']

# Binary_bin
Binary = ['CombinedLoan', 'Age_Map', 'Income_Map']
print("Binary", "\n", Binary)

# Integer_'int'_Ordinal
Integer = ['age', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
           'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents',
           'CombinedDefault']
print("Integer_Ordinal", "\n", Integer)

# Real_'float'_Interval
Real = ['DebtRatio', 'MonthlyIncome', 'NetWorth', 'MonthlyDebtPayments']
print("Real_float", "\n", Real)

**Now for the actual EDA**

I will deal in the order of statistical data types by Binary, Integer, Float.

# 6.2  **UNI-VARITE**


* **BINARY**

In [35]:
"""Binary"""
print("Plotting Bar Plot...for Binary")
# https://stackoverflow.com/questions/35692781/python-plotting-percentage-in-seaborn-bar-plot
sns.set_style("whitegrid")  # Chosen
for col in Binary:
    sns.barplot(x=train_raw_copy[col].value_counts().index, y=train_raw_copy[col].value_counts())
    plt.show()

**Brief Observations**

-'CombinedLoan': Very lob-sided where 3/4 have made loans

-'Age_Map': Data-set contains mostly ages 41 to 63

-'Income_Map': Data-set dominated by High & Mid income earners

* **Integer**


In [36]:
"""Integer"""
print("Plotting Density Plot...for Integer")
# Used as opposed to histogram since this doesnt need bins parameter
i = 0
t1 = train_raw_copy.loc[train_raw_copy['SeriousDlqin2yrs'] != 0]
t0 = train_raw_copy.loc[train_raw_copy['SeriousDlqin2yrs'] == 0]

sns.set_style('whitegrid')
# plt.figure()
fig, ax = plt.subplots(2, 4, figsize=(15, 10))

for feature in Integer:
    i += 1
    plt.subplot(2, 4, i)
    sns.kdeplot(t1[feature], bw=0.5, label="SeriousDlqin2yrs = 1")
    sns.kdeplot(t0[feature], bw=0.5, label="SeriousDlqin2yrs = 0")
    plt.ylabel('Density plot', fontsize=10)
    plt.xlabel(feature, fontsize=10)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=10)
plt.show()

**Brief Observations**

-'age': Same trend as seen from Age_Map in Binary EDA. Majority of data-set are between ages 41 to 63

-'NumberOfTime30-59DaysPastDueNotWorse': Very High Kurtosis & right-skewed

-'NumberOfOpenCreditLinesAndLoans': High Kurtosis & Right-skewed

-'NumberOfTimes90DaysLate': Very High Kurtosis & Very Right-skewed

-'NumberRealEstateLoansOrLines': High Kurtosis & Right-skewed

-'NumberOfTime60-89DaysPastDueNotWorse': Very High Kurtosis & Very Right-skewed

-'NumberOfDependents': High Kurtosis & Right-skewed

-'CombinedDefault': Gaussian distribution shape

Reasonable data-set as those who experience financial distress (SeriousDlqin2yrs=1) have left skewed (Median>Mean).

In other words, data population who experience financial distress have a greater proportion of defaults

**We will switch to a scatter plot for a better visual since, most of the charts have very high kurtosis due to outliers**

In [37]:
print("Plotting Scatter Plot...for Integer")
sns.set_style("whitegrid")  # Chosen
for col in Integer:
    sns.lmplot(y=col, x="Unnamed: 0", data=train_raw_copy, fit_reg=False, hue='SeriousDlqin2yrs', legend=True,
               size=5, aspect=1)
    plt.show()

**Integer Scatter Plot For a Clearer Visual**

**Brief Observations (Improved)**

-'age': Same pattern seen. Concentrated on ages 41 to 63

-'NumberOfTime30-59DaysPastDueNotWorse': Interesting disparity! In other words, we have extreme frequency's.

It's a either or case of Borrowers exceeding the 30-59days

1. very often above 90 times

2. very few times below 15 times

-'NumberOfOpenCreditLinesAndLoans': Evidently, those who have had financial distress (SeriousDlqin2yrs=1) have lower Loans given their poor credit history

-'NumberOfTimes90DaysLate''NumberRealEstateLoansOrLines': Same pattern as 'NumberOfTime30-59DaysPastDueNotWorse'

-'NumberOfTime60-89DaysPastDueNotWorse': Same pattern as 'NumberOfTime30-59DaysPastDueNotWorse'

-'NumberOfDependents': Interesting, those who have had financial distress (SeriousDlqin2yrs=1) tend to have lesser dependents than those who have had financial distress

-'CombinedDefault': n/a

* **Real**

**Similarly, we will switch to a scatter plot**

In [38]:
"""Real"""
print("Plotting Density Plot...for Binary")
# Used as opposed to histogram since this doesnt need bins parameter
i = 0
t1 = train_raw_copy.loc[train_raw_copy['SeriousDlqin2yrs'] != 0]
t0 = train_raw_copy.loc[train_raw_copy['SeriousDlqin2yrs'] == 0]

sns.set_style('whitegrid')
fig, ax = plt.subplots(2, 2, figsize=(8, 6))

for feature in Real:
    i += 1
    plt.subplot(2, 2, i)
    sns.kdeplot(t1[feature], bw=0.5, label="SeriousDlqin2yrs = 1")
    sns.kdeplot(t0[feature], bw=0.5, label="SeriousDlqin2yrs = 0")
    plt.ylabel('Density plot', fontsize=10)
    plt.xlabel(feature, fontsize=10)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=10)
plt.show()

All very high Kurtosis & Right_skewed. As before, We will switch to a scatter plot for a better visual

In [39]:
print("Plotting Scatter Plot...for Real")
sns.set_style("whitegrid")  # Chosen
for col in Real:
    sns.lmplot(y=col, x="Unnamed: 0", data=train_raw_copy, fit_reg=False, hue='SeriousDlqin2yrs', legend=True,
               size=5, aspect=1)
    plt.show()

**Real Scatter Plot For a Clearer Visual**

**Brief Observations (Improved)**

-'DebtRatio': Contrastingly, those who have had financial distress (SeriousDlqin2yrs=1) posses lower DebtRatio

-'MonthlyIncome': Interesting! Data-set actually has a huge income disparity for those who have had financial distress (SeriousDlqin2yrs=1).

Evidently, from our Preliminary Overview it has a StandardDeviation of 3.650860e+04. 

Disparity roughly is between the Less than 33,000 & capping at =100,000. 

(SeriousDlqin2yrs=0) instead is more evened out.

NOTE: Further emphasizes need to implement under sampling to even out the data sample

-'NetWorth': Reasonable sense, since those who have had financial distress (SeriousDlqin2yrs=1) have lower net worth

-'MonthlyDebtPayments': Similar to the pattern found in 'DebtRatio' those who have had financial distress (SeriousDlqin2yrs=1) are paying out lesser existing debts


# 6.3 **BI-VARIATE**

* **Pair Plots**

Note that I have truncated the sample to only 800 to avoid excessive computational costs.

In [40]:
"""Pair-plot"""     
print("Plotting Pair-Plots")
PairPlot = Binary + Integer + Real
# # sample = train_raw_copy.sample(frac=0.5)
sample_SIZE = 800
sample = train_raw_copy.sample(sample_SIZE)
PairPlot.extend(['SeriousDlqin2yrs'])  # Add 'target' into list
var = PairPlot
sample = sample[var]
g = sns.pairplot(sample,  hue='SeriousDlqin2yrs', palette='Set1', diag_kind='kde', plot_kws={"s": 8})
plt.show()
PairPlot.remove('SeriousDlqin2yrs')

Okay we have too many features to fit this space....

No matter, let's individually zoom in by manually splitting them up then!

* **Age_Map Centralized**

In [41]:
"""Age_Map"""
print("Plotting Scatter Plot...for Bi-variate Focus_set_1")
EDA_BiVariate_1 = ['CombinedLoan',
                   'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfDependents', 'CombinedDefault',
                   'DebtRatio', 'MonthlyIncome',  'NetWorth']
sns.set_style("whitegrid")  # Chosen
for col in EDA_BiVariate_1:
    sns.lmplot(y=col, x='age', data=train_raw_copy, fit_reg=False, hue='Age_Map', legend=True,
               size=5, aspect=1)
    plt.show()

**Quick Commentary:**

-'CombinedLoan': For both category's of those who made & did not make a loans, they are dominated by 'Retired' by more than 2x

-'NumberOfTime30-59DaysPastDueNotWorse':

Generic Interpretation>

On the high extreme end, dominated by 'Working' & 'Senior'.

On the low extreme end, dominated by 'Retired'

Specific Interpretation>

On the high extreme end, of exceeding over 95times they are split evenly between 'Working' & 'Senior' category's. But rarely 'Retired'

On the low extreme end, of exceeding over but below 15times they are split evenly between between 'Working' & 'Senior' category's. But this time dominated by 'Retired'

-'NumberOfDependents': 'Working' & 'Senior' category's tend to have higher number of dependents. Evidently, mortality means as we grow older we see more deaths..

-'CombinedDefault': Same pattern as 'CombinedLoan'

-'DebtRatio': Evidently, as 'age' increases 'DebtRatio' increases. But begins falling upon retirement at age 63

-'MonthlyIncome': Similar to before, in Uni-variate EDA we spotted a disparity in 'MonthlyIncome'. But this time we can clearly see a gaussian shape appearing. which is also similar to 'DebtRatio' pattern

Evidently, as 'age' increases 'MonthlyIncome' increases. But begins falling upon retirement at age 63

-'NetWorth': This emphasizes the pattern. Evidently, as 'age' increases 'NetWorth' increases. But begins falling upon retirement at age 63.

HOWEVER, the positive gradient also highlights the flaw in our 'NetWorth' Derivation. Since we are essentially assuming constant income growth & ignoring depreciation considerations for the time value of money (i.e.From my placement year in a Pension's Consultancy firm, we need to include annuities which accounts for both interest rates and mortality.

* **Income_Map Centralized**

In [42]:
"""Income_Map"""
print("Plotting Scatter Plot...for Bi-variate Focus_set_2")
EDA_BiVariate_2 = ['CombinedLoan',
                   'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfDependents', 'CombinedDefault',
                   'DebtRatio', 'MonthlyIncome',  'NetWorth']
sns.set_style("whitegrid")  # Chosen
for col in EDA_BiVariate_1:
    sns.lmplot(y=col, x='MonthlyIncome', data=train_raw_copy, fit_reg=False, hue='Income_Map', legend=True,
               size=5, aspect=1)
    plt.show()

**Quick Commentary:**

-'CombinedLoan': Clearly, those who have made & did not make Loans are dominated by the higher tier income earners

-'NumberOfTime30-59DaysPastDueNotWorse': From a relative perspective, those who exceed the 30-59Days deadline are dominated by higher tier income earners

-'NumberOfDependents': The higher the 'MonthlyIncome', the lower the Dependents

-'CombinedDefault': Same pattern as 'CombinedLoan'

-'DebtRatio': The higher the 'MonthlyIncome', the higher the 'DebtRatio'

-'MonthlyIncome': n/a

-'NetWorth': Obvious of higher 'MonthlyIncome' equates to higher 'NetWorth'

* **Others_A Centralized**

In [43]:
"""Others_A"""
print("Plotting Scatter Plot...for Bi-variate Focus_set_3")
sns.set_style("whitegrid")  # Chosen
sns.lmplot(y='DebtRatio', x='MonthlyIncome', data=train_raw_copy, fit_reg=False, hue='SeriousDlqin2yrs', legend=True,
           size=5, aspect=1)
plt.show()

**Quick Commentary:**

Those who have had financial distress (SeriousDlqin2yrs=1) clearly have lower 'DebtRatio' & 'MonthlyIncome'

* **Others_B Centralized**


In [44]:
"""Others_B"""
sns.lmplot(y='NumberOfTime30-59DaysPastDueNotWorse', x='MonthlyIncome', data=train_raw_copy, fit_reg=False,
           hue='SeriousDlqin2yrs', legend=True, size=5, aspect=1)
plt.show()

**Quick Commentary:**

Contrastingly, often those who have had financial distress (SeriousDlqin2yrs=1) have exceeded the 30-59Days deadline only a few times

* **Others_C Centralized**



In [45]:
"""Others_C"""
sns.lmplot(y='NumberOfOpenCreditLinesAndLoans', x='MonthlyIncome', data=train_raw_copy, fit_reg=False,
           hue='SeriousDlqin2yrs', legend=True, size=5, aspect=1)
plt.show()

**Quick Commentary:**

Similarly, those who have had financial distress (SeriousDlqin2yrs=1) actually open lesser loans than those of without financial distress (SeriousDlqin2yrs=0)

* **Others_D Centralized**


In [46]:
"""Others_D"""
sns.lmplot(y='NumberRealEstateLoansOrLines', x='MonthlyIncome', data=train_raw_copy, fit_reg=False,
           hue='SeriousDlqin2yrs', legend=True, size=5, aspect=1)
plt.show()

**Quick Commentary:**

Same pattern as **Others_C_Centralized** !!

* **Others_E Centralized**

In [47]:
"""Others_E"""
sns.lmplot(y='DebtRatio', x='NumberRealEstateLoansOrLines', data=train_raw_copy, fit_reg=False, hue='SeriousDlqin2yrs',
           legend=True, size=5, aspect=1)
plt.show()

**Quick Commentary:**

Similarly, those who have had financial distress (SeriousDlqin2yrs=1) actually a lower 'DebtRatio' than those of without financial distress (SeriousDlqin2yrs=0)

* **Others_F Centralized**

In [48]:
"""Others_F"""
sns.lmplot(y='NumberOfOpenCreditLinesAndLoans', x='NumberRealEstateLoansOrLines', data=train_raw_copy, fit_reg=False,
           hue='SeriousDlqin2yrs', legend=True, size=5, aspect=1)
plt.show()

**Quick Commentary:**

A realistic scenario here.
As the RealEstateLoans increases, borrowers tendency to open other lines and loans (Credit loans) decreases.

# **7.EDA Commentary**

# 7.1 Summative

This summarizes the thorough EDA commentary above.

**Focus_set_1**

-Disobedient acts (making excessive loans & defaults or exceed deadlines) often made by 'Retired'

-Realistic data-set, income plateau and falls while proceeding with age

**Focus_set_2**

-Disobedient acts often made by Higher-tier income

-Credit balloon; since as Higher-tier income exhibit higher extremes of making loans & experiencing default

-Higher-tier income tend to have lesser dependents. Self-centred data-set?

**Focus_set_3**

-Contrasting relationships. When experiencing financial distress (SeriousDlqin2yrs=1), borrows actually have a "apt" financial circumstance (lower 'DebtRatio' & exceeding deadlines & making loans or having defaults) but only have low 'MonthlyIncome'

Alternative POV, low 'MonthlyIncome' main driver for financial distress, while 'DebtRatio' & exceeding deadlines & making loans or having defaults play less significant effect.


# 7.2 **Heat-Map Reference**

Let's let the math tell us more using the Heat-Map

In [49]:
cor = train_raw_copy.corr()
plt.figure(figsize=(17, 10))
# sns.set(font_scale=0.7)
sns.heatmap(cor, annot=True, cmap='YlGn')
plt.show()

# **8.Final Feature Engineering Commentary**

Now with the added support of the HeatMap correlations to our EDA analysis, we can further justify and decide on which features to discard or keep. 

**Joining No# Times past due: CombinedDefault**

-Keep CombinedDefault since it outperforms in correlation

-Keep 'NumberOfTime30-59DaysPastDueNotWorse' since it has the highest 'target variable' correlation amonst the original 3. But still bears low multi-collinearity with CombinedDefault

-Drop NumberOfTime60-89DaysPastDueNotWorse & NumberOfTimes90DaysLate

**New Feature: Net Worth**

-Keep NetWorth since it outperforms in correlation

-Drop MonthlyIncome; Since NetWorth has higher correlation. NetWorth as a proxy for 'MonthlyIncome' given the formula.

**Join No# Loans: CombinedLoans**

-Keep CombinedLoans. Used as proxy for original features

-Drop NumberOfOpenCreditLinesAndLoans & NumberRealEstateLoansOrLines to avoid multi-collinearity. 

**New Feature: Monthly debt payments**

-Drop MonthlyDebtPayments since its correlation is still lower than 'DebtRatio'

**New Feature: Age Map (is_Retired & is_Senior & is_Working**

-Drop Age_Map & is_Retired & is_Senior & is_Working, since original 'age' outperforms

**New Feature: Income Map (is_LowY & is_MidY & is_HighY**

-Drop Income_Map & is_LowY & is_MidY & is_HighY, since original 'MonthlyIncome'' outperforms



# ***NOTE: This Kernel primarily focuses on Feature Engineering & EDA (Chapters 3 to 8). Hence, from here forth it is for brief comparative & closure of this Kernel purposes.***


# **9.Prepare Data A**


# 9.1 Establish Features Decided on (Keep/Discard)

Now to set the features to keep for modelling!

* Keep or Discard respective features

In [50]:
"""Prepare Data"""

"""Dummy set to be used by Model to Validate above drop"""
train_raw_copy_DropValidate = train_raw_copy

"""Dropping"""
ColumnsToDrop = ['Unnamed: 0', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate',
                 'MonthlyIncome',
                 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines',
                 'MonthlyDebtPayments',
                 'Age_Map', 'is_Retired', 'is_Senior', 'is_Working',
                 'Income_Map', 'is_LowY', 'is_MidY', 'is_HighY']

train_raw_copy.drop(columns=ColumnsToDrop, inplace=True)

* Check Again

In [51]:
"""Check Again"""
basic_details(train_raw_copy)

# 9.2 **Check Heat-Map Again**

Great, we only have a maximum multi-collinearity correlation of 0.31

In [52]:
"""Correlation Heat-Map"""
cor = train_raw_copy.corr()
plt.figure(figsize=(10, 8))
# sns.set(font_scale=0.7)
sns.heatmap(cor, annot=True, cmap='YlGn')
plt.show()

All good! Maximum of 0.31 correlations are acceptable

# 9.3 Replicate for Test set

Now to replicate all that we have done for the Test_Set 

And also checking the shape!

In [53]:
"""Repeat Feature Engineering for Test Set"""


def clean_dataset(dataset):

    """CombinedDefault"""
    dataset['CD'] = (dataset['NumberOfTime30-59DaysPastDueNotWorse']
                     + dataset['NumberOfTimes90DaysLate']
                     + dataset['NumberOfTime60-89DaysPastDueNotWorse'])
    dataset['CombinedDefault'] = 1
    dataset.loc[(dataset['CD'] == 0), 'CombinedDefault'] = 0
    del dataset['CD']

    """NetWorth"""
    dataset['NetWorth'] = dataset['MonthlyIncome'] * dataset['age'] / NetWorthDivisor

    """CombinedLoans"""
    dataset['CL'] = (dataset['NumberOfOpenCreditLinesAndLoans']
                     + dataset['NumberRealEstateLoansOrLines'])
    dataset['CombinedLoan'] = 1
    dataset.loc[dataset['CL'] >= LoanLinesBuffer, 'CombinedLoan'] = 1
    dataset.loc[dataset['CL'] < LoanLinesBuffer, 'CombinedLoan'] = 0
    del dataset['CL']

    """Dropping"""
    to_drop = ['Unnamed: 0', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate',
               'MonthlyIncome',
               'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines']

    dataset.drop(columns=to_drop, inplace=True)


clean_dataset(test_raw)


"""Check Columns"""
print(train_raw_copy.columns)
print(test_raw.columns)
"""Check Shape"""
print(train_raw_copy.shape)
print(test_raw.shape)

# **10.Quick Feature Engineering Validation**

Now just a precautionary measure here. We are going to use L1 regularized LASSO to do a quick rough validation of the features that we have chosen to exclude.

We are simply comparing the LASSO accuracy scores before VS after the drop. 

Ideally, we should see that we have no difference in accuracy scores. Hence, indicating that the drop had no effect on the model accuracy.

# 10.1 Accuracy Score (Without Preliminary Engineering)

In [54]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
############# PRE DROPPING FEATURES
train_x1 = train_raw_copy_DropValidate.drop("SeriousDlqin2yrs", axis=1).copy()
#Y1 = train_raw_copy_DropValidate.SeriousDlqin2yrs
Y1 = train_raw['SeriousDlqin2yrs'].values

# Preparing train/test split of dataset            
X_train, X_validation, y_train, y_validation = train_test_split(train_x1, Y1, train_size=0.9, random_state=1234)

##### Instantiate Logistic Regression 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Transform data for LogRef fitting"""
scaler = StandardScaler()
std_data = scaler.fit_transform(X_train.values)

# Establish Model
RandomState=42
model_LogRegLASSO1 = LogisticRegression(penalty='l1', C=0.1, random_state=RandomState, solver='liblinear', n_jobs=1)
model_LogRegLASSO1.fit(std_data, y_train)

In [55]:
# Run Accuracy score without any dropping of features
print("PRE DROPPING FEATURES: Running LASSO Accuracy Score without features drop...")
# make predictions for test data and evaluate
y_pred = model_LogRegLASSO1.predict(X_validation)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_validation, predictions)
print("PRE Accuracy: %.2f%%" % (accuracy * 100.0))

# 10.2 Accuracy Score (With Preliminary Engineering)

In [56]:
############# POST DROPPING FEATURES
train_x2 = train_raw_copy.drop("SeriousDlqin2yrs", axis=1).copy()   
Y2 = train_raw_copy['SeriousDlqin2yrs'].values  

# Preparing train/test split of dataset            
X_train, X_validation, y_train, y_validation = train_test_split(train_x2, Y2, train_size=0.9, random_state=1234)

##### Instantiate Logistic Regression 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Transform data for LogRef fitting"""
scaler = StandardScaler()
std_data = scaler.fit_transform(X_train.values)

# Establish Model

model_LogRegLASSO1 = LogisticRegression(penalty='l1', C=0.1, random_state=RandomState, solver='liblinear', n_jobs=1)
model_LogRegLASSO1.fit(std_data, y_train)

In [57]:
# Run Accuracy score without any dropping of features
print("POST DROPPING FEATURES: Running LASSO Accuracy Score with features dropped...")
# make predictions for test data and evaluate
y_pred = model_LogRegLASSO1.predict(X_validation)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_validation, predictions)
print("POST Accuracy: %.2f%%" % (accuracy * 100.0))

**Great both 93.03% and henceno difference in accuracy scores!**

**Let's proceed with the actual ensemble modelling then!**

# **11.Prepare Data - B**

# 11.1**Train/Test Split**

In [58]:
"""
Prepare Data
"""
# Split our predictors and the target variable in our data-sets
"""Train set"""
X = train_raw_copy.drop("SeriousDlqin2yrs", axis=1).copy()
y = train_raw_copy.SeriousDlqin2yrs
print(X.shape, '\n', y.shape)

"""Test set"""
X_test = test_raw.drop("SeriousDlqin2yrs", axis=1).copy()
y_test = test_raw.SeriousDlqin2yrs
print(X_test.shape, '\n', y_test.shape)


"""Train/test split of data-set"""
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, random_state=1234)

More or less same shapes, lets proceed!

# **Models**

Here I am going to use:

White-Box Models (LASSO, Ridge, Logit Balanced)
Black-Box Models (XGB Classifier, Random Forest Classifier)

We will first define a general function to help us normalize all feature scorings. 

As white box models coefficients, white black box models use GINI importance

# 11.2 Feature Comparison Function

In [59]:
"""
Model
"""
RandomState = 42

"""Preparing Side to Side Comparative Function"""
from sklearn.preprocessing import MinMaxScaler


def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks))


names = X.columns
ranks = {}
print('Prep done...')

# 11.3 **LASSO L1 Regularized**



* Establish Model *Unhide to view output

In [60]:
"""
LASSO via LogisticRegression l1 penalty - Feature Importance - PART 1
"""
print('Running LASSO via LogisticRegression l1 penalty...')
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

""""Transform data for LogRef fitting"""
scaler = StandardScaler()
std_data = scaler.fit_transform(X_train.values)

"""Establish Model"""
model_LogRegLASSO = LogisticRegression(penalty='l1', C=0.1, random_state=RandomState, solver='liblinear', n_jobs=1)
model_LogRegLASSO.fit(std_data, y_train)

"""For Side To Side"""
ranks["LogRegLASSO"] = rank_to_dict(list(map(float, model_LogRegLASSO.coef_.reshape(len(names), -1))), names, order=1)
print(ranks["LogRegLASSO"])

* Plot Feature Rankings

In [61]:
"""Plotting"""
import operator
listsLASSO = sorted(ranks["LogRegLASSO"].items(), key=operator.itemgetter(1))
# convert list>array>dataframe
dfLASSO = pd.DataFrame(np.array(listsLASSO).reshape(len(listsLASSO), 2),
                       columns=['Features', 'Ranks']).sort_values('Ranks')
dfLASSO['Ranks'] = dfLASSO['Ranks'].astype(float)

dfLASSO.plot.bar(x='Features', y='Ranks', color='blue')
plt.xticks(rotation=90)
plt.show()

# 11.4 **Ridge L2 Regularized**



* Establish Model *Unhide to view output

In [62]:
"""
Ridge via LogisticRegression l2 penalty - Feature Importance - PART 1
"""

print('Running Ridge via LogisticRegression l2 penalty...')
"""Establish Model"""
model_LogRegRidge = LogisticRegression(penalty='l2', C=0.1, random_state=RandomState, solver='liblinear', n_jobs=1)
model_LogRegRidge.fit(std_data, y_train)

"""For Side To Side"""
ranks["LogRegRidge"] = rank_to_dict(list(map(float, model_LogRegRidge.coef_.reshape(len(names), -1))),
                                    names, order=1)
print(ranks["LogRegRidge"])

* Plot Feature Rankings

In [63]:
"""Plotting"""
import operator
listsRidge = sorted(ranks["LogRegRidge"].items(), key=operator.itemgetter(1))
dfRidge = pd.DataFrame(np.array(listsRidge).reshape(len(listsRidge), 2),
                       columns=['Features', 'Ranks']).sort_values('Ranks')
dfRidge['Ranks'] = dfRidge['Ranks'].astype(float)

dfRidge.plot.bar(x='Features', y='Ranks', color='blue')
plt.xticks(rotation=90)
plt.show()

# 11.5 **Logit Balanced by sample weight**



* Establish Model *Unhide to view output

In [64]:
"""
LogisticRegression Balanced - Feature Importance - PART 1
"""
print('RunningLogisticRegression Balanced...')
"""Establish Model"""
model_LogRegBalance = LogisticRegression(class_weight='balanced', C=0.1, random_state=RandomState, solver='liblinear', n_jobs=1)
model_LogRegBalance.fit(std_data, y_train)

"""For Side To Side"""
ranks["LogRegBalance"] = rank_to_dict(list(map(float, model_LogRegBalance.coef_.reshape(len(names), -1))),
                                      names, order=1)
print(ranks["LogRegBalance"])

* Plot Feature Rankings

In [65]:
"""Plotting"""
import operator
listsBal = sorted(ranks["LogRegBalance"].items(), key=operator.itemgetter(1))
dfBal = pd.DataFrame(np.array(listsBal).reshape(len(listsBal), 2),
                     columns=['Features', 'Ranks']).sort_values('Ranks')
dfBal['Ranks'] = dfBal['Ranks'].astype(float)

dfBal.plot.bar(x='Features', y='Ranks', color='blue')
plt.xticks(rotation=90)
plt.show()

# 11.6 **XGB Classifier**

* Establish Model *Unhide to view output

In [66]:
"""
XGBClassifier - Feature Importance - PART 1
"""
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance
print("Running XGBClassifier Feature Importance Part 1...")
model_XGBC = XGBClassifier(objective='binary:logistic',
                           max_depth=7, min_child_weight=5,
                           gamma=0,
                           learning_rate=0.1, n_estimators=100,)
# model_XGBC.fit(X_train, y_train)
model_XGBC.fit(std_data, y_train)
print("XGBClassifier Fitted")

"""Side To Side"""
print("Ranking Features with XGBClassifier...")
ranks["XGBC"] = rank_to_dict(model_XGBC.feature_importances_, names)
print(ranks["XGBC"])

* Plot Feature Rankings

In [67]:
"""Plotting"""
# plot feature importance for feature selection using default inbuild function
plot_importance(model_XGBC)
plt.show()

# 11.7 **Random Forest Classifier**

* Establish Model *Unhide to view output

In [68]:
"""
Random Forest Classifier - Feature Importance - PART 1
"""
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model_RFC = RandomForestClassifier(bootstrap=True, max_depth=80,
                                   criterion='entropy',
                                   min_samples_leaf=3, min_samples_split=10, n_estimators=100)
# model_RFC.fit(X_train, y_train)
model_RFC.fit(std_data, y_train)
print("Random Forest Classifier Fitted")

"""Side To Side"""
print("Ranking Features with RFClassifier...")
ranks["RFC"] = rank_to_dict(model_RFC.feature_importances_, names)
print(ranks["RFC"])

* Plot Feature Rankings

In [69]:
"""Plotting"""
# For Chart
importance = pd.DataFrame({'feature': X_train.columns, 'importance': np.round(model_RFC.feature_importances_, 3)})
importance_sorted = importance.sort_values('importance', ascending=False).set_index('feature')
# plot feature importance for feature selection using default inbuild function
#print(importance_sorted)
importance_sorted.plot.bar()
plt.show()

# **12.Features (Side To Side Comparison)**

Here i present 2 different approaches

# 12.1 (Coefficient Values) Quick Easy Method

First quick approach allows for quick printing if you are short of time.

In [70]:
"""
Collate Side to Side df - Feature Importance
"""

"""Quick Print Method"""
# Create empty dictionary to store the mean value calculated across all the scores
r = {}
for name in names:
    # This is the alternative rounding method from the earlier map & lambda combination
    r[name] = round(np.mean([ranks[method][name] for method in ranks.keys()]), 2)

methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")

print("\t%s" % "\t".join(methods))
for name in names:
    print("%s\t%s" % (name, "\t".join(map(str, [ranks[method][name] for method in methods]))))

As you easily see this is quick but alignments are off....Now for a neater and more presentable method

# 12.2 (Coefficient Values) Neat DataFrame Method

Second approach you could loop through and procure a dataframe for further use going forward.

In [71]:
"""Alternatively, Set into a df"""
# Loop through dictionary of scores to append into a dataframe
row_index = 0
AllFeatures_columns = ['Feature', 'Scores']
AllFeats = pd.DataFrame(columns=AllFeatures_columns)
for name in names:
    AllFeats.loc[row_index, 'Feature'] = name
    AllFeats.loc[row_index, 'Scores'] = [ranks[method][name] for method in methods]

    row_index += 1

# Here the dataframe scores are a list in a list.
# To split them, we convert the 'Scores' column from a dataframe into a list & back into a dataframe again
AllFeatures_only = pd.DataFrame(AllFeats.Scores.tolist(), )
# Now to rename the column headers
AllFeatures_only.rename(columns={0: 'LogRegBalance', 1: 'LogRegLASSO', 2: 'LogRegRidge',
                                 3: 'Random ForestClassifier', 4: 'XGB Classifier', 5: 'Mean'}, inplace=True)
AllFeatures_only = AllFeatures_only[['LogRegBalance', 'LogRegLASSO', 'LogRegRidge',
                                     'Random ForestClassifier', 'XGB Classifier', 'Mean']]
# Now to join both dataframes
AllFeatures_compare = AllFeats.join(AllFeatures_only).drop(['Scores'], axis=1)
display(AllFeatures_compare)

# **Plotting**

Now likewise plotting I've given 2 methods

# 12.3 (Plotting) Quick Method

A quick plot to see the rough trend.

In [72]:
"""Quick Plotting"""
# Plotting
df = AllFeatures_compare.melt('Feature', var_name='cols',  value_name='vals')
g = sns.factorplot(x="Feature", y="vals", hue='cols', data=df, size=10, aspect=2)

plt.xticks(rotation=90)
plt.show()

# 12.4 (Plotting) Sorted Neat Method

A better plot IMO could be used, where we sort by the mean first. Similar, to what the Excel Pivot Chart does.

In [73]:
"""Sorted Plotting"""
AllFeatures_compare_sort = AllFeatures_compare.sort_values(by=['Mean'], ascending=True)
order_ascending = AllFeatures_compare_sort['Feature']
#Plotting
df2 = AllFeatures_compare_sort.melt('Feature', var_name='cols',  value_name='vals')
# ONLY Difference is that now we use row_order to sort based on the above ascending Ascending Mean Features
# g2 = sns.factorplot(x="Feature", y="vals", hue='cols', data=df2, size=10, aspect=2, row_order=order_ascending)
g2 = sns.factorplot(x="Feature", y="vals", hue='cols', data=df2, size=10, aspect=2, row_order=order_ascending)
plt.xticks(rotation=60)
plt.show()

# 12.5 Quick Commentary Result (Feature Engineering VS Importance)

* **Group_A (Combine Feature)**

-Quantity of Defaults

Wow, it was ranked highest!! Even outdid its component "NumberOfTime30-59DaysPastDueNotWorse", But on the downside ranked the lowest by XGB.

Note we discarded its other 2 components: "NumberOfTimes90DaysLate" & "NumberOfTime60-89DaysPastDueNotWorse"

* **Group_B (New Feature Net Worth)**

-Net Worth

Woho!! it was ranked in the middle. Good call on this one :)

* **Group_C (Combine Feature)**

-Combined Loan

Looks like our this new feature we created ranked last....Even it's component "RevolvingUtilizationOfUnsecuredLines" outdid it...Bad call hahaha

* **New Feature (Debt Payments)**

Note we discarded this

* **New Feature (Age Category)**

Note we discarded this

* **New Feature (Social Economic Status SES Category)**

Note we discarded this


# **13.ROC AUC (Side To Side Comparison)**

Now as before we will collate the ROC & AUC for each model & plot all of them for comparison

In [74]:
"""
Collate Side to Side df - ROC AUC
"""
# Ensemble Comparison of ROC AUC
from sklearn import model_selection
import matplotlib.pyplot as plt


print("Charting ROC AUC for Ensembles...")
from sklearn.metrics import roc_curve, auc

# Establish Models
models = [
    {
        'label': 'LASSO',
        'model': model_LogRegLASSO,
    },
    {
        'label': 'Ridge',
        'model': model_LogRegRidge,
    },
    {
        'label': 'LogReg Balance',
        'model': model_LogRegBalance,
    },
    {
        'label': 'XGBoost Classifier',
        'model': model_XGBC,
    },
    {
        'label': 'Random Forest Classifier',
        'model': model_RFC,
    }
]

# Models Plot-loop
for m in models:
    #fpr, tpr, thresholds = roc_curve(y_validation, m['model'].predict_proba(X_validation).T[1])
    scaler = StandardScaler()
    std_data2 = scaler.fit_transform(X_validation.values)
    fpr, tpr, thresholds = roc_curve(y_validation, m['model'].predict_proba(std_data2).T[1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], roc_auc))

# Set Plotting attributes
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc=0, fontsize='small')
plt.show()

# **14.Cross Validation (Side To Side Comparison)**

Now as before we will collate the relevant 'Cross Validated' (CV) accuracy scores for each model & compare all of them.

In [66]:
"""Collate Side to Side df - Accuracy Scores"""
# run model 10x with 60/30 split, but intentionally leaving out 10% avoiding overfitting
cv_split = model_selection.ShuffleSplit(n_splits=10, test_size=.3, train_size=.6, random_state=0)

# Ensemble Comparison of Accuracy Scores
# Set dataframe for appending``
# pd.options.display.max_columns = 100
ACCScores_columns = ['Model Name', 'Train Accuracy Mean', 'Test Accuracy Mean']
ACCScores_compare = pd.DataFrame(columns=ACCScores_columns)

# Models CV-loop
row_index = 0
for m in models:
    # Name of Model
    ACCScores_compare.loc[row_index, 'Model Name'] = m['label']

    # Execute Cross Validation (CV)
    cv_results = model_selection.cross_validate(m['model'], X_train, y_train, cv=cv_split)
    # Model Train Accuracy
    ACCScores_compare.loc[row_index, 'Train Accuracy Mean'] = cv_results['train_score'].mean()
    # Model Test Accuracy
    ACCScores_compare.loc[row_index, 'Test Accuracy Mean'] = cv_results['test_score'].mean()

    row_index += 1

display(ACCScores_compare)

Thank you for reading till the end! If you like more commentary on this tail chapters, do refer to my other Kernel  "Ensemble Models Comparison Techniques".

Hope you now have a better understanding of EDA & Feature Engineering!!

Cheers